In [2]:
import pandas as pd
import numpy as np

In [3]:
def read_biosample_data(file='data/biospecimenDataFileRepo_20220228.xlsx'):
    df = pd.read_excel(file)
    return df
def read_patient_data(file="data/clinicalDataFileRepo_20220228.xlsx"):
    return pd.read_excel(file)
def read_file_data(file="data/file-table.tsv"):
    return pd.read_csv(file,sep='\t')
biosamples = read_biosample_data()
patients = read_patient_data()
files = read_file_data()

/home/ochapman/miniconda3/envs/py3/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/home/ochapman/miniconda3/envs/py3/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [4]:
# How many tumor WGS files do we have?
malignant_keywords=['Tumor','tumor','metastasis','Primary Blood Derived Cancer - Bone Marrow',
                    'Blood Derived Cancer - Bone Marrow, Post-treatment','Primary Blood Derived Cancer - Peripheral Blood',
                   'relapse','Blood Derived Cancer - Peripheral Blood, Post-treatment','metastasis or relapse']
foi = files[files["Tissue Type (Source Text)"].isin(malignant_keywords)]
print("Tumor WGS files:",len(foi))

Tumor WGS files: 2745


In [5]:
# How many patients does this correspond to?
boi = foi["Participants ID"].unique()
print("Tumor patients for which WGS exists:",len(boi))

Tumor patients for which WGS exists: 1557


In [6]:
# How many biosamples does this correspond to?
boi = foi["Biospecimen ID"].unique()
print("Tumor biospecimens for which WGS exists:",len(boi))

Tumor biospecimens for which WGS exists: 1785


In [7]:
# Which study codes do we need access to?
soi = foi["Study Code"].unique()
#soi = soi[~((soi=="PBTA-CBTN") | (soi=="PBTA-PNOC"))]
print(soi)

['KF-NBL' 'PBTA-CBTN' 'KF-CHDALL' 'PBTA-PNOC' 'KF-OS' 'TARGET-AML'
 'TARGET-NB' 'KF-FALL']


In [12]:
# How many biosamples per study?
for s in soi:
    print(s,len(foi[foi["Study Code"] == s]["Biospecimen ID"].unique()))

KF-NBL 338
PBTA-CBTN 968
KF-CHDALL 252
PBTA-PNOC 66
KF-OS 93
TARGET-AML 55
TARGET-NB 6
KF-FALL 7


In [9]:
def get_additional_files():
    patients = foi[(foi["Study Code"].isin(soi))]["Participants ID"].unique()
    tumor_files = foi[(foi["Participants ID"].isin(patients))].sort_values(["Participants ID","Biospecimen ID","File Format"])
    print(len(tumor_files))
    tumor_files = tumor_files.drop_duplicates(subset=["Participants ID","Biospecimen ID","File Format"],keep='last')
    print(len(tumor_files))
    
    nonmalignant_keywords=['Normal','normal','Fibroblasts from Bone Marrow Normal','Non-Tumor']
    normal_files = files[(files["Tissue Type (Source Text)"].isin(nonmalignant_keywords)) &
                        (files["Participants ID"].isin(tumor_files["Participants ID"]))]
    normal_files = normal_files.sort_values(["Participants ID","Biospecimen ID","File Format"])
    print(len(normal_files))
    normal_files = normal_files.drop_duplicates(["Participants ID","Biospecimen ID","File Format"],keep='last')
    print(len(normal_files))
    return tumor_files,normal_files
tumor_files, normal_files = get_additional_files()


774
758
487
486


In [12]:
# tumor_files.to_csv('tumor_files.tsv',sep='\t',index=False)
# normal_files.to_csv('normal_files.tsv',sep='\t',index=False)
# Note: these are aligned to different references. Should probably break it up by project.

In [13]:
tumor_files

,File ID,Participants ID,Study Code,File Format,File Name,Sample External ID,Biospecimen ID,Tissue Type (Source Text),Diagnosis (Source Text)
9529,GF_HVS1T3G0,PT_0001K4K1,KF-CHDALL,cram,s3://kf-study-us-east-1-prd-sd-z6mwd3h0/source...,P6159_T,BS_Z93M9TXC,Tumor,"Down syndrome, Acute lymphoblastic leukemia"
11358,GF_NEN4V1A1,PT_00FB6H4C,KF-CHDALL,cram,s3://kf-study-us-east-1-prd-sd-z6mwd3h0/source...,P10597_T,BS_AGA1ZH2Z,Tumor,"Acute lymphoblastic leukemia, Down syndrome"
13194,GF_RYBSK13X,PT_00QYKRAX,KF-NBL,bam,SL253434.bam,GMKF-30-PASTHB03-01A-01D,BS_CG9KDK95,Tumor,"Neuroblastoma, Neuroblastoma, Neuroblastoma, N..."
1727,GF_389HPBZB,PT_00Y8C0XA,KF-NBL,bam,SL262747.bam,GMKF-30-PAUDEK03-01A-01D,BS_4ZE1R1FW,Tumor,"Neuroblastoma, Neuroblastoma, Neuroblastoma, N..."
5430,GF_A4AR6CHK,PT_027AV1N1,KF-CHDALL,cram,s3://kf-study-us-east-1-prd-sd-z6mwd3h0/source...,P11313_T,BS_PFKHGVMR,Tumor,"Down syndrome, Acute lymphoblastic leukemia"
...,...,...,...,...,...,...,...,...,...
12293,GF_Q6GYH4J1,PT_ZSZRCE81,KF-CHDALL,cram,s3://kf-study-us-east-1-prd-sd-z6mwd3h0/source...,P10825_T,BS_T40GZNCJ,Tumor,"Acute lymphoblastic leukemia, Down syndrome"
4470,GF_8BNQE78X,PT_ZT2NW6WA,KF-NBL,bam,SL264173.bam,GMKF-30-PAUCNL03-01A-01D,BS_KTHF096D,Tumor,"Neuroblastoma, Neuroblastoma, Neuroblastoma, N..."
8145,GF_F6QK3EQK,PT_ZVXBKPC5,KF-CHDALL,cram,s3://kf-study-us-east-1-prd-sd-z6mwd3h0/source...,P11354_T,BS_EWH47999,Tumor,"Acute lymphoblastic leukemia, Down syndrome"
15278,GF_WVB1F2Z3,PT_ZW22K0YF,KF-NBL,bam,SL262722.bam,GMKF-30-PASZTC03-01A-01D,BS_DGV11G69,Tumor,"Neuroblastoma, Neuroblastoma, Neuroblastoma, N..."


In [1]:
def prettyprint_files(df):
    string = ', '.join(df["File ID"])
    return string

In [14]:
prettyprint_files(tumor_files)

'GF_HVS1T3G0, GF_NEN4V1A1, GF_RYBSK13X, GF_389HPBZB, GF_A4AR6CHK, GF_C2ARZ67J, GF_24T6V28F, GF_5NE9XD2V, GF_S9MSCWSE, GF_HH9C5DYX, GF_FPST0WPE, GF_P8X607RQ, GF_DJ7404AM, GF_WAXP3WTE, GF_G6NN88FQ, GF_GWWHESX1, GF_52P1DDMG, GF_CG088HKD, GF_CEBS4FD1, GF_FB4XB50G, GF_PRTCVAEM, GF_X9G3F9GG, GF_CPYM5XYG, GF_EQWAVWVK, GF_4EJW664M, GF_ABVK4VQA, GF_9Y7A2Z43, GF_NSMCSRF8, GF_10XKEP2A, GF_W93F5HNS, GF_T6TVKCHR, GF_4BKKS2J9, GF_F7GSSZW4, GF_5EK27W5Z, GF_AV896TT6, GF_PHMEJEPK, GF_0HCRMKEA, GF_VKZW08ZB, GF_EEM7D4TA, GF_17H3AH3Z, GF_W1MWQ5FP, GF_47C1H4EB, GF_PT3XJAZZ, GF_8CY002Y0, GF_R5W35HEP, GF_YB33WZWN, GF_THRP1AFG, GF_YZEB59ES, GF_5P2P45Z7, GF_32NXA0X2, GF_MFAWYTCS, GF_86K1YE5P, GF_EKHXB01D, GF_13WSRAXQ, GF_20G3TV17, GF_M11A9X01, GF_JDD0PNHA, GF_MKEJ5ZWF, GF_EZ4KE4CR, GF_VVXJWV9M, GF_4PN971E7, GF_92Z3B85P, GF_54DJB75E, GF_0WVQ0YCC, GF_69D6RTTE, GF_ANCH2W6G, GF_K0Y6BXQQ, GF_HBBX1PVA, GF_MK4AH78C, GF_P5MN7A2Y, GF_5Q595TEQ, GF_27T5YZEC, GF_EQXVBCZP, GF_TFMKZT5M, GF_Q891KN9W, GF_MEWQNPEZ, GF_YFPD6PCQ